In [1]:
# === 0. Imports & config ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
import joblib
import shap
from tqdm import tqdm
import os

# display settings
pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 200)
sns.set(style="whitegrid")
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


ModuleNotFoundError: No module named 'seaborn'

In [ ]:
import pandas as pd

DATA_PATH = "/content/drive/MyDrive/DA project/ecommerce_customer_data_custom_ratios.csv"

# Load CSV first without parsing dates
df = pd.read_csv(DATA_PATH)

# List possible purchase date column names
possible_date_cols = ["Purchase Date", "Purchase_Date", "purchase_date"]

# Find which column exists in the CSV
date_cols = [c for c in possible_date_cols if c in df.columns]

# Parse only the existing column(s)
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Normalize all column names to lowercase + underscores
df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

# Check columns
print("Columns after normalization:", df.columns.tolist())
df.head()

In [ ]:
# === 2. Basic cleaning & required column mapping ===
# We expect at least: customer_id, purchase_date, total_purchase_amount
# Try common alternative names
col_map = {}
if "customer_id" not in df.columns:
    # attempt common alternatives
    for alt in ["customerid", "customer", "cust_id"]:
        if alt in df.columns:
            col_map[alt] = "customer_id"
if "purchase_date" not in df.columns:
    for alt in ["purchase_date", "purchase_date", "order_date", "date"]:
        if alt in df.columns:
            col_map[alt] = "purchase_date"
if "total_purchase_amount" not in df.columns and "order_amount" not in df.columns and "total_amount" not in df.columns:
    for alt in ["total_purchase_amount", "order_amount", "total_amount", "price", "product_price"]:
        if alt in df.columns:
            col_map[alt] = "total_purchase_amount"

# Apply renames if found
if col_map:
    df = df.rename(columns=col_map)
    print("Renamed cols:", col_map)

# Ensure essential columns exist
required = ["customer_id", "purchase_date", "total_purchase_amount"]
for r in required:
    if r not in df.columns:
        raise ValueError(f"Required column '{r}' not found in dataset. Found columns: {df.columns.tolist()}")

# Ensure data types
df["purchase_date"] = pd.to_datetime(df["purchase_date"])
df["total_purchase_amount"] = pd.to_numeric(df["total_purchase_amount"], errors="coerce").fillna(0.0)
print("Dataset shape:", df.shape)

In [ ]:
# === 3. Quick EDA (optional) ===
print("Number of unique customers:", df["customer_id"].nunique())
print("Date range:", df["purchase_date"].min(), "to", df["purchase_date"].max())
print()
display(df.sample(5))
print()
print("Checking Duplicate value")
df.duplicated()

In [ ]:
# === 4. Define cutoff (observation) date and prediction horizon ===
# Choose cutoff so that you have at least prediction_horizon days after it in the data
# By default we'll use last date minus 365 days as cutoff to allow 12-month future window available.
max_date = df["purchase_date"].max()
prediction_horizon_days = 365  # predict next 12 months
cutoff_date = max_date - pd.Timedelta(days=prediction_horizon_days)
print("Using cutoff_date (end of observation):", cutoff_date.date(),
      " — prediction horizon days:", prediction_horizon_days)

In [ ]:
# === 5. Split history and future (target) windows ===
history = df[df["purchase_date"] <= cutoff_date].copy()
future = df[(df["purchase_date"] > cutoff_date) & (df["purchase_date"] <= cutoff_date + pd.Timedelta(days=prediction_horizon_days))].copy()
print("History rows:", len(history), "Future rows:", len(future))

In [ ]:
# === 6. Build target: future spend per customer in prediction window ===
target = future.groupby("customer_id")["total_purchase_amount"].sum().rename("future_spend_12m").reset_index()
# include customers who appear in history but have zero future spend
customers_history = history["customer_id"].unique()
target = pd.DataFrame({"customer_id": customers_history}).merge(target, on="customer_id", how="left")
target["future_spend_12m"] = target["future_spend_12m"].fillna(0.0)
print("Target rows:", target.shape[0])
target.head()

In [ ]:
# === 7. Feature engineering on history window ===
# We'll create RFM + behavioral + demographic features
# Required columns we may use: product_category, quantity, returns, customer_age, gender, payment_method

# Basic RFM
agg_funcs = {
    "purchase_date": ["min", "max", "count"],
    "total_purchase_amount": ["sum", "mean", "std"]
}
# include quantity if present
if "quantity" in history.columns:
    agg_funcs["quantity"] = ["sum", "mean"]
if "returns" in history.columns:
    agg_funcs["returns"] = ["sum"]

hist_agg = history.groupby("customer_id").agg(agg_funcs)
# flatten columns
hist_agg.columns = ["_".join(col).strip() for col in hist_agg.columns.values]
hist_agg = hist_agg.reset_index()
# rename
hist_agg = hist_agg.rename(columns={
    "purchase_date_min": "first_purchase_date",
    "purchase_date_max": "last_purchase_date",
    "purchase_date_count": "num_orders",
    "total_purchase_amount_sum": "total_spent",
    "total_purchase_amount_mean": "avg_order_value",
    "total_purchase_amount_std": "std_order_value"
})
# fill std NaN with 0
hist_agg["std_order_value"] = hist_agg["std_order_value"].fillna(0.0)

# recency and tenure
hist_agg["recency_days"] = (cutoff_date - hist_agg["last_purchase_date"]).dt.days
hist_agg["tenure_days"] = (hist_agg["last_purchase_date"] - hist_agg["first_purchase_date"]).dt.days.clip(lower=1)

# frequency per month
hist_agg["orders_per_month"] = hist_agg["num_orders"] / (hist_agg["tenure_days"] / 30.0)

# quantity features
if "quantity_sum" in hist_agg.columns and "num_orders" in hist_agg.columns:
    hist_agg["avg_items_per_order"] = hist_agg["quantity_mean"].fillna(0.0)
else:
    hist_agg["avg_items_per_order"] = 0.0

# returns ratio
if "returns_sum" in hist_agg.columns:
    hist_agg["returns_ratio"] = hist_agg["returns_sum"] / hist_agg["num_orders"]
else:
    hist_agg["returns_ratio"] = 0.0

# product category diversity
if "product_category" in history.columns:
    cat_div = history.groupby("customer_id")["product_category"].nunique().rename("prod_category_count").reset_index()
    hist_agg = hist_agg.merge(cat_div, on="customer_id", how="left")
    hist_agg["prod_category_count"] = hist_agg["prod_category_count"].fillna(0).astype(int)
else:
    hist_agg["prod_category_count"] = 0

# preferred payment method (mode)
if "payment_method" in history.columns:
    pref_pay = history.groupby("customer_id")["payment_method"].agg(lambda x: x.mode().iat[0] if len(x.mode())>0 else np.nan).rename("preferred_payment_method").reset_index()
    hist_agg = hist_agg.merge(pref_pay, on="customer_id", how="left")
else:
    hist_agg["preferred_payment_method"] = np.nan

# demographics: age, gender (take most recent or mode)
if "customer_age" in history.columns:
    age = history.groupby("customer_id")["customer_age"].median().rename("customer_age").reset_index()
    hist_agg = hist_agg.merge(age, on="customer_id", how="left")
else:
    hist_agg["customer_age"] = np.nan

if "gender" in history.columns:
    gender = history.groupby("customer_id")["gender"].agg(lambda x: x.mode().iat[0] if len(x.mode())>0 else np.nan).rename("gender").reset_index()
    hist_agg = hist_agg.merge(gender, on="customer_id", how="left")
else:
    hist_agg["gender"] = np.nan

# Fill missing numeric demographics
hist_agg["customer_age"] = hist_agg["customer_age"].fillna(hist_agg["customer_age"].median(skipna=True))
hist_agg[["returns_ratio"]] = hist_agg[["returns_ratio"]].fillna(0.0)

print("Feature frame shape:", hist_agg.shape)
hist_agg.head(7)

In [ ]:
# === 8. Merge features with target & prepare model dataframe ===
data = hist_agg.merge(target, on="customer_id", how="left")
data["future_spend_12m"] = data["future_spend_12m"].fillna(0.0)

# drop date columns that leak
data = data.drop(columns=[c for c in ["first_purchase_date", "last_purchase_date"] if c in data.columns])

print("Model dataset shape:", data.shape)
data.head()

In [ ]:
# === 9. Prepare X (features) and y (target). Handle categorical features ===
# Choose feature columns
exclude = {"customer_id", "future_spend_12m"}
feature_cols = [c for c in data.columns if c not in exclude]
print("Features used:", feature_cols)

X = data[feature_cols].copy()
y = data["future_spend_12m"].copy()

# Many targets are heavy-tailed; use log1p transform for modeling stability
y_log = np.log1p(y)

# Simple categorical encoding: one-hot for payment_method and gender (small cardinality)
cat_cols = [c for c in ["preferred_payment_method", "gender"] if c in X.columns]
num_cols = [c for c in X.columns if c not in cat_cols + ["customer_id"]]
print("Numeric cols:", num_cols)
print("Categorical cols:", cat_cols)

# One-hot encode categorical
X_encoded = pd.get_dummies(X, columns=cat_cols, dummy_na=True, drop_first=True)
# Fill any remaining NaNs with 0
X_encoded = X_encoded.fillna(0.0)
print("Encoded feature shape:", X_encoded.shape)

In [ ]:
# === 10. Train/validation/test split ===
X_train, X_temp, y_train, y_temp = train_test_split(X_encoded, y_log, test_size=0.3, random_state=RANDOM_STATE)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=RANDOM_STATE)

print(X_train.shape, X_val.shape, X_test.shape)

In [ ]:
# === 11. Train XGBoost regressor with early stopping ===
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    "objective": "reg:squarederror",
    "learning_rate": 0.05,
    "max_depth": 6,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "seed": RANDOM_STATE,
    "verbosity": 1
}

evals = [(dtrain, "train"), (dval, "val")]
model = xgb.train(params, dtrain, num_boost_round=2000, early_stopping_rounds=50, evals=evals, verbose_eval=50)
print("Best iteration:", model.best_iteration)

In [ ]:
# === 12. Evaluation helpers and metrics (convert back from log) ===
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import xgboost as xgb

def evaluate_model(mdl, X, y_log_true, label="set"):
    # Convert features to DMatrix
    d = xgb.DMatrix(X)

    # Predict
    y_log_pred = mdl.predict(d)

    # Convert from log back to actual scale
    y_true = np.expm1(y_log_true)
    y_pred = np.expm1(y_log_pred)

    # Metrics
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))  # <-- fixed here
    r2 = r2_score(y_true, y_pred)

    print(f"{label} -> MAE: {mae:.2f}, RMSE: {rmse:.2f}, R2: {r2:.3f}")
    return y_true, y_pred

# Evaluate on validation and test sets
y_true_val, y_pred_val = evaluate_model(model, X_val, y_val, "Validation")
y_true_test, y_pred_test = evaluate_model(model, X_test, y_test, "Test")


In [ ]:
# === 13. Business metric: Top-decile capture ===
def topk_capture(y_true, y_pred, k=0.1):
    n = len(y_pred)
    topk = int(np.ceil(n * k))
    idx = np.argsort(-y_pred)[:topk]  # top predicted
    captured = y_true[idx].sum() / y_true.sum() if y_true.sum() > 0 else 0.0
    return captured

print("Top-10% capture (validation):", topk_capture(np.array(y_true_val), np.array(y_pred_val), 0.1))
print("Top-10% capture (test):", topk_capture(np.array(y_true_test), np.array(y_pred_test), 0.1))

In [ ]:
# === 14. Save model and training artifacts ===
os.makedirs("artifacts", exist_ok=True)
joblib.dump(model, "artifacts/xgb_ltv_model.joblib")
# Save feature columns order and training median (if needed)
joblib.dump(X_encoded.columns.tolist(), "artifacts/feature_columns.joblib")
print("Saved model and feature list to artifacts/")

In [ ]:
# === 15. Create final predictions for all customers and save CSV ===
d_all = xgb.DMatrix(X_encoded)
pred_log_all = model.predict(d_all)
pred_all = np.expm1(pred_log_all)
out = pd.DataFrame({
    "customer_id": data["customer_id"],
    "predicted_ltv_12m": pred_all,
    "historical_total_spent": data["total_spent"],
    "num_orders": data["num_orders"]
})
out.to_csv("predicted_ltv_12m.csv", index=False)
print("Saved predicted_ltv_12m.csv (rows):", len(out))
out.head()

In [ ]:
# === 16. SHAP explanations (global & example local) ===
# Note: SHAP can be slow for large models/data. Do for sample or top features.
explainer = shap.Explainer(model)
# compute shap values on a sample to save time
sample_idx = np.random.choice(X_encoded.shape[0], min(500, X_encoded.shape[0]), replace=False)
X_sample = X_encoded.iloc[sample_idx]
shap_values = explainer(X_sample)

# summary plot (global)
shap.summary_plot(shap_values, X_sample, show=True)

# to display top features numerically
importance_df = pd.DataFrame({
    "feature": X_sample.columns,
    "mean_abs_shap": np.abs(shap_values.values).mean(axis=0)
}).sort_values("mean_abs_shap", ascending=False).head(20)
display(importance_df)

In [ ]:
# === 17. Visualizations ===
# Actual vs Predicted (test)
plt.figure(figsize=(8,6))
plt.scatter(y_true_test, y_pred_test, alpha=0.4)
plt.plot([0, max(y_true_test.max(), y_pred_test.max())],[0, max(y_true_test.max(), y_pred_test.max())], color='red', linewidth=1)
plt.xlabel("Actual future 12m spend")
plt.ylabel("Predicted future 12m spend")
plt.title("Actual vs Predicted (Test)")
plt.xscale("symlog")
plt.yscale("symlog")
plt.show()

In [ ]:
# Distribution of predicted LTV
plt.figure(figsize=(8,4))
sns.histplot(out["predicted_ltv_12m"], bins=50, kde=False)
plt.title("Distribution of Predicted LTV (12m)")
plt.show()

In [ ]:
# Cumulative revenue capture by decile
deciles = out.copy()
deciles["pred_pct_rank"] = deciles["predicted_ltv_12m"].rank(pct=True)
deciles = deciles.sort_values("predicted_ltv_12m", ascending=False)
deciles["cum_actual"] = deciles["historical_total_spent"].cumsum()
deciles["cum_actual_pct"] = deciles["cum_actual"] / deciles["historical_total_spent"].sum()
plt.figure(figsize=(8,4))
plt.plot(np.arange(len(deciles))/len(deciles), deciles["cum_actual_pct"])
plt.xlabel("Fraction of customers (sorted by predicted LTV desc)")
plt.ylabel("Cumulative historical revenue captured")
plt.title("Lift curve (by predicted LTV)")
plt.show()

In [ ]:
# === 18. Export model summary & README snippet (quick) ===
with open("artifacts/model_info.txt", "w") as f:
    f.write("XGBoost LTV model\n")
    f.write(f"Trained on: {pd.Timestamp.now()}\n")
    f.write(f"Cutoff date: {cutoff_date}\n")
    f.write(f"Prediction horizon days: {prediction_horizon_days}\n")
    f.write("Features used:\n")
    for c in X_encoded.columns:
        f.write(f"- {c}\n")
print("Wrote artifacts/model_info.txt")

In [ ]:
import pandas as pd
import joblib
from google.colab import files

# Make sure X_test is the customer-level features for test set
# Reset index to get customer_id column if needed
X_test_reset = X_test.reset_index()  # 'customer_id' must be part of index
if "customer_id" not in X_test_reset.columns:
    # If customer_id is in index, reset_index ensures it becomes a column
    X_test_reset = X_test_reset.rename(columns={"index": "customer_id"})

# Create predictions dataframe
predictions_df = pd.DataFrame({
    "customer_id": X_test_reset["customer_id"],
    "predicted_ltv_12m": y_pred_test  # Predictions from evaluate_model
})

# Save CSV
predictions_df.to_csv("predicted_ltv_12m.csv", index=False)
files.download("predicted_ltv_12m.csv")

# Save trained model
joblib.dump(model, "xgb_ltv_model.joblib")
files.download("xgb_ltv_model.joblib")

print("predictions CSV and trained model saved successfully!")
